# Path Integral Monte Carlo Pipeline for the 1D Anharmonic Oscillator

This notebook is a complete end-to-end workflow for the quartic oscillator in Euclidean time,
with Hamiltonian

$$H = \frac{p^2}{2m} + \frac{1}{2}m\omega^2 x^2 + \lambda x^4$$

We use Path Integral Monte Carlo (PIMC) to sample periodic lattice paths and estimate:
- translationally averaged correlators $G(\tau)$,
- effective mass plateaus, including bootstrap uncertainty bands,
- and the lowest energy gap extracted from a weighted cosh fit.

The notebook is organized to run top-to-bottom with no manual edits beyond the **Parameters** cell.
Invalid or low-information effective-mass points are intentionally represented as `NaN` and masked in plots by default.


## Installation Reminder

Use Python **3.10+** in the notebook kernel.

Recommended setup (outside this notebook):

```bash
python -m pip install -r requirements.txt
python -m pip install -e .
```

The import cell also auto-adds `src/` from the repository root when possible,
so the notebook can run even if editable install is missing.


In [ ]:
# =============================
# Parameters (edit only this cell)
# =============================

RUN_NAME = "notebook_pipeline"
OUT_ROOT = None  # Defaults to REPO_ROOT; set a path string to override.

# Physical parameters
MASS = 1.0
OMEGA = 1.0

# Lattice parameters
BETA = 4.0
N_SLICES = 128

# Sampler parameters
SAMPLER_METHOD = "hmc"  # "hmc" (recommended) or "metropolis_rb"
N_THERM = 1000
N_SWEEPS = 6000
MEASURE_EVERY = 5
PROPOSAL_WIDTH = 1.0  # Used by metropolis_rb
TUNE = True
TUNE_INTERVAL = 25
TARGET_ACCEPT = 0.75
HMC_STEP_SIZE = 0.055
HMC_N_LEAPFROG = 16
HMC_MASS = 1.0
SEED = 123
SHOW_PROGRESS = False

# Analysis parameters
BIN_SIZE = 12  # Fixed default for reproducibility.
AUTO_BIN_FROM_PILOT = False  # Optional pilot-based auto-binning.
PILOT_BIN_SIZE = 6
N_BOOT = 180
SUBTRACT_MEAN = False
FIT_TAU_MIN_FRAC = 0.2
FIT_TAU_MAX_FRAC = 0.5
SYMMETRIZE_CORRELATOR = True
EFFECTIVE_MASS_INVALID_AS_NAN = True
# Mask invalid/low-information points in Delta_eff by default.
EFFECTIVE_MASS_SNR_MIN = None

# Couplings: include harmonic validation point first.
LAMBDAS = [0.0, 0.25, 0.5, 1.0]


## Run Harmonic Case + Analysis + Plots

This section uses masked effective masses (NaN for invalid arcosh ratios or low-SNR points)
and overlays the bootstrap 16-84% band when available.


In [ ]:
import sys
from dataclasses import replace
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display


def _find_repo_root(start: Path) -> Path:
    for candidate in (start, *start.parents):
        if (candidate / "src").is_dir():
            return candidate
    raise RuntimeError("Could not locate repository root containing 'src/'")


REPO_ROOT = _find_repo_root(Path.cwd().resolve())
src_path = REPO_ROOT / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

if OUT_ROOT in (None, "", "."):
    OUT_ROOT = REPO_ROOT
else:
    OUT_ROOT = Path(OUT_ROOT).expanduser().resolve()

from pimc_oscillator.config import LatticeParams, PotentialParams, RunConfig, SamplerParams
from pimc_oscillator.experiment import run_lambda_scan, run_single_case
from pimc_oscillator.plotting import (
    plot_correlator,
    plot_effective_mass,
    plot_gap_vs_lambda,
    plot_histogram_with_gaussian,
)

try:
    plt.style.use("seaborn-v0_8-whitegrid")
except (OSError, ValueError):
    plt.style.use("default")

REPORTS_DIR: Path | None = None


def save_and_display(fig: plt.Figure, filename: str) -> Path:
    if REPORTS_DIR is None:
        raise RuntimeError("REPORTS_DIR is not set. Run the scan cell first.")
    path = REPORTS_DIR / filename
    fig.savefig(path, dpi=180, bbox_inches="tight")
    display(fig)
    return path


print(f"Repository root: {REPO_ROOT}")
print(f"Output root: {OUT_ROOT}")


In [ ]:
base_sampler = SamplerParams(
    method=SAMPLER_METHOD,
    n_therm=N_THERM,
    n_sweeps=N_SWEEPS,
    measure_every=MEASURE_EVERY,
    proposal_width=PROPOSAL_WIDTH,
    tune=TUNE,
    tune_interval=TUNE_INTERVAL,
    target_accept=TARGET_ACCEPT,
    hmc_step_size=HMC_STEP_SIZE,
    hmc_n_leapfrog=HMC_N_LEAPFROG,
    hmc_mass=HMC_MASS,
    seed=SEED,
)

base_config = RunConfig(
    potential=PotentialParams(m=MASS, omega=OMEGA, lam=0.0),
    lattice=LatticeParams(beta=BETA, n_slices=N_SLICES),
    sampler=base_sampler,
    run_name=RUN_NAME,
)

if BIN_SIZE is None and not AUTO_BIN_FROM_PILOT:
    raise ValueError("Set BIN_SIZE to an integer or enable AUTO_BIN_FROM_PILOT.")

active_bin_size = int(BIN_SIZE) if BIN_SIZE is not None else 10
if AUTO_BIN_FROM_PILOT:
    pilot_sampler = replace(
        base_sampler,
        n_therm=max(400, N_THERM // 3),
        n_sweeps=max(1200, N_SWEEPS // 3),
        seed=SEED + 9090,
    )
    pilot_config = replace(
        base_config,
        sampler=pilot_sampler,
        run_name=f"{RUN_NAME}_pilot",
    )
    pilot = run_single_case(
        pilot_config,
        out_root=OUT_ROOT,
        bin_size=PILOT_BIN_SIZE,
        n_boot=min(120, N_BOOT),
        subtract_mean=SUBTRACT_MEAN,
        fit_tau_min_frac=FIT_TAU_MIN_FRAC,
        fit_tau_max_frac=FIT_TAU_MAX_FRAC,
        symmetrize_correlator=SYMMETRIZE_CORRELATOR,
        effective_mass_invalid_as_nan=EFFECTIVE_MASS_INVALID_AS_NAN,
        effective_mass_snr_min=EFFECTIVE_MASS_SNR_MIN,
        progress=SHOW_PROGRESS,
    )
    active_bin_size = max(2, pilot["suggested_bin_size"])
    print(
        f"Pilot diagnostics (lambda=0): tau_int={pilot['tau_int_x2']:.2f}, "
        f"ESS~{pilot['effective_sample_size_x2']:.1f}."
    )
    print(f"Auto-selected BIN_SIZE = {active_bin_size}")
else:
    print(f"Using fixed BIN_SIZE = {active_bin_size}")

scan_result = run_lambda_scan(
    LAMBDAS,
    base_config=base_config,
    out_root=OUT_ROOT,
    run_name=RUN_NAME,
    seed_stride=1000,
    bin_size=int(active_bin_size),
    n_boot=N_BOOT,
    subtract_mean=SUBTRACT_MEAN,
    fit_tau_min_frac=FIT_TAU_MIN_FRAC,
    fit_tau_max_frac=FIT_TAU_MAX_FRAC,
    symmetrize_correlator=SYMMETRIZE_CORRELATOR,
    effective_mass_invalid_as_nan=EFFECTIVE_MASS_INVALID_AS_NAN,
    effective_mass_snr_min=EFFECTIVE_MASS_SNR_MIN,
    progress=SHOW_PROGRESS,
)

REPORTS_DIR = Path(scan_result["paths"]["reports_dir"])
results_by_lambda = scan_result["results_by_lambda"]
lambda_sorted = scan_result["lambda_sorted"]
gaps = scan_result["gaps"]
gap_errs = scan_result["gap_errs"]
harmonic_result = results_by_lambda[0.0]

print(f"Run ID: {scan_result['run_id']}")
print(f"Logs dir: {scan_result['paths']['logs_dir']}")
print(f"Reports dir: {scan_result['paths']['reports_dir']}")

print(
    f"Harmonic diagnostics: acceptance={harmonic_result['acceptance_rate']:.3f}, "
    f"tau_int(<x^2>)={harmonic_result['tau_int_x2']:.2f}, "
    f"ESS~{harmonic_result['effective_sample_size_x2']:.1f} / "
    f"{harmonic_result['samples'].shape[0]}"
)
print(
    f"Suggested BIN_SIZE from diagnostics: {harmonic_result['suggested_bin_size']} "
    f"(currently using {active_bin_size})."
)


In [ ]:
tau = harmonic_result["tau"]
g_exact = harmonic_result["G_exact"]
harmonic_eff_boot = harmonic_result.get("effective_mass_bootstrap")

fig_corr_h = plot_correlator(
    tau=tau,
    G=harmonic_result["G"],
    G_err=harmonic_result["G_err"],
    G_exact=g_exact,
    title="Harmonic case (lambda = 0): correlator vs exact",
)
save_and_display(fig_corr_h, "harmonic_correlator_vs_exact.png")

harmonic_delta_curve = (
    harmonic_eff_boot.mean
    if harmonic_eff_boot is not None
    else harmonic_result["effective_mass"]
)

fig_eff_h = plot_effective_mass(
    tau_mid=harmonic_result["effective_mass_tau"],
    Delta_eff=harmonic_delta_curve,
    Delta_eff_q16=None if harmonic_eff_boot is None else harmonic_eff_boot.q16,
    Delta_eff_q84=None if harmonic_eff_boot is None else harmonic_eff_boot.q84,
    fit_window=harmonic_result["fit_window_tau"],
    title="Harmonic case: effective mass",
)
ax_h = fig_eff_h.axes[0]
ax_h.axhline(OMEGA, color="black", linestyle="--", linewidth=1.2, label="exact omega")
ax_h.axhline(
    harmonic_result["gap_bootstrap"].delta_mean,
    color="tab:red",
    linestyle=":",
    linewidth=1.4,
    label="bootstrap gap",
)
ax_h.legend(loc="best")
save_and_display(fig_eff_h, "harmonic_effective_mass.png")

x2_exact = harmonic_result["x2_exact"]
fig_hist_h = plot_histogram_with_gaussian(
    x_values=harmonic_result["samples"].ravel(),
    mean=0.0,
    var=x2_exact if x2_exact is not None else harmonic_result["x2_bootstrap_mean"],
    title="Harmonic case: sampled x histogram with analytic Gaussian",
)
save_and_display(fig_hist_h, "harmonic_histogram_gaussian_overlay.png")

print(
    "<x^2> (bootstrap): "
    f"{harmonic_result['x2_bootstrap_mean']:.6f} +/- {harmonic_result['x2_bootstrap_std']:.6f}"
)
print(f"<x^2> exact (harmonic): {x2_exact:.6f}")
print(
    "Extracted gap (bootstrap): "
    f"{harmonic_result['gap_bootstrap'].delta_mean:.6f} "
    f"+/- {harmonic_result['gap_bootstrap'].delta_std:.6f}"
)
print(f"Expected harmonic gap omega: {OMEGA:.6f}")


## Run Anharmonic Cases + Analysis + Plots

Effective-mass curves below prefer bootstrap means and show the 16-84% band.
Masked NaN points indicate invalid or low-information late-time estimates.


In [ ]:
for lam in lambda_sorted:
    lam_value = float(lam)
    if lam_value == 0.0:
        continue

    result = results_by_lambda[lam_value]
    eff_boot = result.get("effective_mass_bootstrap")
    delta_curve = eff_boot.mean if eff_boot is not None else result["effective_mass"]

    fig_corr = plot_correlator(
        tau=result["tau"],
        G=result["G"],
        G_err=result["G_err"],
        title=f"Anharmonic case (lambda = {lam_value:.2f}): correlator",
    )
    save_and_display(fig_corr, f"anharmonic_lam_{lam_value:.2f}".replace(".", "p") + "_correlator.png")

    fig_eff = plot_effective_mass(
        tau_mid=result["effective_mass_tau"],
        Delta_eff=delta_curve,
        Delta_eff_q16=None if eff_boot is None else eff_boot.q16,
        Delta_eff_q84=None if eff_boot is None else eff_boot.q84,
        fit_window=result["fit_window_tau"],
        title=f"Anharmonic case (lambda = {lam_value:.2f}): effective mass",
    )
    ax = fig_eff.axes[0]
    ax.axhline(
        result["gap_bootstrap"].delta_mean,
        color="tab:red",
        linestyle="--",
        linewidth=1.4,
        label="bootstrap gap",
    )
    ax.legend(loc="best")
    save_and_display(
        fig_eff,
        f"anharmonic_lam_{lam_value:.2f}".replace(".", "p")
        + "_effective_mass.png",
    )

fig_gap = plot_gap_vs_lambda(
    lams=lambda_sorted,
    gaps=gaps,
    gap_errs=gap_errs,
    title="Extracted gap vs quartic coupling lambda",
)
save_and_display(fig_gap, "gap_vs_lambda.png")


## Summary

In [ ]:
header = f"{'lambda':>8} {'gap':>12} {'error':>12} {'fit_gap':>12} {'accept':>10} {'tau_int':>10} {'ESS':>10}"
print(header)
print("-" * len(header))

for lam in lambda_sorted:
    result = results_by_lambda[float(lam)]
    print(
        f"{lam:8.2f} "
        f"{result['gap_bootstrap'].delta_mean:12.6f} "
        f"{result['gap_bootstrap'].delta_std:12.6f} "
        f"{result['gap_fit'].delta:12.6f} "
        f"{result['acceptance_rate']:10.4f} "
        f"{result['tau_int_x2']:10.3f} "
        f"{result['effective_sample_size_x2']:10.1f}"
    )

trend_ok = bool(np.all(np.diff(gaps) >= -np.maximum(gap_errs[1:], 1e-12)))
print()
print(f"Gap trend check (allowing ~1 sigma fluctuations): {trend_ok}")
print(f"Saved run summary: {scan_result['paths']['run_summary_path']}")
print(f"All figures and JSON outputs are in: {scan_result['paths']['reports_dir']}")
